# Imports

In [109]:
from langchain_ollama import OllamaLLM # Para levantar el modelo LLama3.1
from langchain_community.document_loaders import PyMuPDFLoader #Para Cargar los PDFs
from langchain.text_splitter import RecursiveCharacterTextSplitter # Para hacer los CHUNKS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings # Para hacer el embeddings de los CHUNKS
from langchain_community.vectorstores import Chroma # Para trabajar con la BD vectorial
from langchain.prompts import PromptTemplate # Para hacer el template del prompt del modelo LLama
from langchain.chains import RetrievalQA # Para hacer las preguntas al modelo y obtener las respuestas
from langchain_core.documents.base import Document # Para hacer la concatenación de documentos

In [110]:
import os

# Parámetros

In [111]:
CHUNK_SIZE=2000
CHUNK_OVERLAP=500

# Consideraciones
* Se tomó la decisión de que todos los documentos van en una sola colección

# Modelo
* El modelo es el llama3.1
* Debe de estar installado el modelo y ollama.
* [Descargar Ollama](https://ollama.com/)
* Descargar el modelo en la terminal luego de descargar OLLAMA `ollama pull llama3.1` **Pesa 4GB**


In [112]:
llm = OllamaLLM(model="llama3.1")
llm.invoke("Hola, quien eres?")

'Hola! Soy un modelo de lenguaje artificial entrenado para comunicarme con humanos en español. No tengo una identidad personal ni soy un ser humano real, sino más bien una herramienta diseñada para ayudar y proporcionar información a través de conversaciones como esta.\n\nMe llamo "Chat" y estoy aquí para responder tus preguntas, ofrecer consejos, jugar contigo o simplemente charlar sobre cualquier tema que te interese. ¿En qué puedo ayudarte hoy?'

# Cargar los Documentos (RAG)

In [113]:
loaders=[]
direc = './PDF'

for name in os.listdir(direc):
    full_path = os.path.join(direc, name)
    loader= PyMuPDFLoader(full_path)
    loaders.append(loader)

for loader in loaders:
    print(loader.file_path)


./PDF\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf
./PDF\Reglamento-permanencia-UCN.Nuevo_.-D4-2023-salida.pdf


In [114]:
pdfs=[]
for loader in loaders:
    pdf=loader.load()
    pdfs.append(pdf)
print(pdfs[0])
print(pdfs[1])

[Document(metadata={'source': './PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'file_path': './PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 0, 'total_pages': 31, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241020235819Z', 'trapped': ''}, page_content='Universidad \nCatólica del Norte \nREGLAMENTO GENERAL \nDE DOCENCIA DE PREGRADO \nJUNIO 2024 \n'), Document(metadata={'source': './PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'file_path': './PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 1, 'total_pages': 31, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241020235819Z', 'trapped': ''}, page_content='ÍNDICE \nTÍTULO \n| Disposiciones generale \nTÍTULO II De la calidad de estudiante \nTÍTULO IIl Admi

### Unir los documentos en uno solo(Etapa de prueba)

In [115]:
# Combinar los contenidos de los documentos
# contenido_combinado = "\n\n".join([pdf.page_content for pdf in pdfs])

AttributeError: 'list' object has no attribute 'page_content'

In [ ]:
# combined_metadata = {
#     'source': ', '.join({pdf.metadata['source'] for pdf in pdfs}),
#     'total_pages': sum(pdf.metadata['total_pages'] for pdf in pdfs),
#     'format': ', '.join({pdf.metadata['format'] for pdf in pdfs}),
#     # Otros campos combinados según necesites
# }

# Splitter

In [116]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

### Hacer los Chunks

In [ ]:
chunks=[]
for pdf in pdfs:
    pdf_chunks= text_splitter.split_documents(pdf)
    chunks.append(pdf_chunks)
print("***********Primer Doc**********",chunks[0][0])
print("***********Segundo Doc**********",chunks[1][0])

In [ ]:
chunks[0]

In [ ]:
chunks[1]

# Hacer el embedding

## Obtener el modelo de embedding desde HuggingFace

In [120]:
embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


# Chroma


### Creación de la base de datos

- Solo se hizo el embedding del documento general de docencia de pregrado(Hay que arreglar concatenar los documentos)

In [121]:
vs = Chroma.from_documents(
    documents=chunks[0],
    embedding=embed_model,
    persist_directory="chroma_db_dir",  # Local mode with in-memory storage only
    collection_name="reglamentos_UCN"
)

In [122]:
vectorstore = Chroma(embedding_function=embed_model,
                     persist_directory="./chroma_db_dir/",
                     collection_name="reglamentos_UCN")

In [123]:
retriever=vectorstore.as_retriever(search_kwargs={'k': 10})

### Hacer el Template Del Promt

In [142]:
from langchain.prompts import PromptTemplate

custom_prompt_template = """ 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo
Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.

Contexto: {context}
Pregunta: {question}

responde siempre en español a no ser que te pidan lo contrario
Respuesta útil:
"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

In [143]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs={"prompt": prompt})

## Prompts De prueba

In [149]:
response = qa.invoke({"query": "Cuales son los prerrequisitos de una asignatura?"})
response['result']
response

{'query': 'Cuales son los prerrequisitos de una asignatura?',
 'result': 'Según el texto proporcionado, los prerrequisitos para cursar una asignatura son:\n\n* Tener inscrita al menos 2 asignaturas por semestre en curso.\n* No quedar con menos de 12 SCT luego de la renuncia.\n* Tener inscrita en primera o segunda oportunidad aquella(s) actividad(es) a la(s) cual(es) desea renunciar.\n* No haber renunciado con anterioridad a la(s) misma(s) actividad(es).\n* No estar cursando la(s) actividad(es) en régimen de Tutoría.',
 'source_documents': [Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': './PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': 'D:20241020235819Z', 'page': 18, 'producer': 'iLovePDF', 'source': './PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'subject': '', 'title': '', 'total_pages': 31, 'trapped': ''}, page_content='Los requisitos para realizar la solicitud son: \na) \